# Landsat LST Sharpening Example - Interactive

In [1]:
import ee
import folium

import openet.lst

# ee.Authenticate()
ee.Initialize()

viridis_palette = ['#440154', '#433982', '#30678D', '#218F8B', '#36B677', '#8ED542', '#FDE725']

image_size = 768


In [2]:
landsat_id = 'LC08_042034_20180705'
landsat_img = ee.Image(f'LANDSAT/LC08/C02/T1_L2/{landsat_id}')

map_xy = [-120, 37]
# map_xy = landsat_sr_img.geometry().centroid().getInfo()['coordinates']
# print(map_xy)

max_zoom = 13


In [3]:
# "Prep" the Landsat Collection 2 Level 2 image
# The "SPACECRAFT_ID" property must be set on the image for the algorithm to work
input_bands = ee.Dictionary({
    'LANDSAT_5': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'ST_B6', 'QA_PIXEL'],
    'LANDSAT_7': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'ST_B6', 'QA_PIXEL'],
    'LANDSAT_8': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'QA_PIXEL'],
    'LANDSAT_9': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'QA_PIXEL'],
})
output_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'lst', 'qa']
spacecraft_id = ee.String(landsat_img.get('SPACECRAFT_ID'))
prep_img = (
    landsat_img
    .select(input_bands.get(spacecraft_id), output_bands)
    .multiply([0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.00341802, 1])
    .add([-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, 149.0, 1])
    .set({'system:index': landsat_img.get('system:index'),
          'system:time_start': landsat_img.get('system:time_start'),
          'system:id': landsat_img.get('system:id'), 
          'SPACECRAFT_ID': spacecraft_id,
         })
)

# Run the sharpening algorithm
output_img = openet.lst.Model(prep_img).sharpen()


In [4]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr="Map Data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
        show=show
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [5]:
my_map = folium.Map(location=[map_xy[1], map_xy[0]], zoom_start=max_zoom, height=image_size)

# Add the layers to the map
my_map.add_ee_layer(
    landsat_img.select([3, 2, 1]).multiply(0.0000275).add(-0.2), 
    {'min': 0, 'max': 0.3}, 'True Color', False)
# my_map.add_ee_layer(
#     landsat_img.select(['ST_B10']).multiply(0.00341802).add(149.0), 
#     {'min': 295, 'max': 325, 'palette': viridis_palette}, 'Thermal Raw', False)
my_map.add_ee_layer(
    prep_img.select(['lst']), 
    {'min': 295, 'max': 325, 'palette': viridis_palette}, 'Thermal Original', False)
my_map.add_ee_layer(
    output_img.select(['lst_sharpened']), 
    {'min': 295, 'max': 325, 'palette': viridis_palette}, 'Thermal Sharpened', True)
my_map.add_ee_layer(
    prep_img.select(['lst']).subtract(output_img.select(['lst_sharpened'])), 
    {'min': -2, 'max': 2, 'palette': ['ff0000', 'ffffff', '0000ff']}, 'Difference', True)

my_map.add_child(folium.LayerControl())
display(my_map)